In [1]:
import pandas as pd
import pickle
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae

In [2]:
df = pd.read_csv("data/daily_metrics_2011_2013.csv")
df = df.fillna(0).round(2)

with open('parrot.pkl', 'rb') as f:
    dates = pickle.load(f)
df["Date"] = dates
df_ = pd.read_csv('data/Fiyatveri.csv',sep=";")

In [3]:
def reverser(x):
    m = x.split(".")
    if len(m[0])>1:
        a = m[2]+"-"+m[1]+"-"+m[0]
    else:
        a = m[2]+"-"+m[1]+"-"+"0"+m[0]
    return a
def numerica(x):
    if type(x) == type(""):
        return x.replace(",", "", 1).replace(",", ".", 1)
def voll(x):
    return x.replace("-","0")

In [4]:
df

,average_clustering,density,number_of_edges,number_of_nodes,degree_assortativity_coefficient,Date
0,0.00,0.50,1,2,0.00,2009-01-09
1,0.00,0.09,10,11,-0.61,2009-01-12
2,0.00,0.03,10,20,0.00,2009-01-13
3,0.00,0.02,20,32,0.00,2009-01-14
4,0.00,0.02,21,32,-0.15,2009-01-15
...,...,...,...,...,...,...
1814,0.01,0.00,94086,62942,-0.03,2013-12-29
1815,0.01,0.00,100129,65118,-0.03,2013-12-30
1816,0.01,0.00,85026,55280,-0.03,2013-12-31
1817,0.01,0.00,68290,45349,-0.02,2014-01-01


In [5]:
df_["Date"] = df_["Date"].apply(reverser)

In [6]:
totaldf  = df_.merge(df,how="left",on="Date")

In [7]:
totaldf["Open"] =totaldf["Open"].apply(numerica)
totaldf["High"] =totaldf["High"].apply(numerica)
totaldf["Low"] = totaldf["Low"].apply(numerica) 
totaldf["Open"] = pd.to_numeric(totaldf["Open"])
totaldf["High"] = pd.to_numeric(totaldf["High"])
totaldf["Low"] = pd.to_numeric(totaldf["Low"])
totaldf["Vol"] = totaldf["Vol"].apply(voll)
totaldf["Vol"] = pd.to_numeric(totaldf["Vol"])
totaldf["Change %"] = pd.to_numeric(totaldf["Change %"])

In [8]:
totaldf= totaldf[["Price"]]

In [9]:
totaldf

,Price
0,0.2
1,0.2
2,0.2
3,0.2
4,0.3
...,...
1151,802.0
1152,803.0
1153,762.0
1154,785.0


In [10]:
def model_creator(X_train):
    regressor = Sequential()
    regressor.add(LSTM(units = 50  , return_sequences = True , input_shape = (X_train.shape[1] , X_train.shape[2])))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50  , return_sequences = True  ))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50  , return_sequences = True  ))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50 ))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units= 1))
    regressor.compile(optimizer='adam', loss='mean_squared_error')
    return regressor

In [11]:
sc= MinMaxScaler(feature_range = (0 , 1))
totaldf_scaled =sc.fit_transform(totaldf)

X_train = []
y_train =[]
x_test = []
for i in range (60,1000):
    X_train.append(totaldf_scaled[i-60:i])
    y_train.append(totaldf_scaled[i, 0])
for i in range(1000,1156):
    x_test.append(totaldf_scaled[i-60:i])
    
X_train , y_train, x_test =np.array(X_train), np.array(y_train), np.array(x_test)

X_train =np.reshape(X_train,(X_train.shape[0] , X_train.shape[1] ,  X_train.shape[2] ))

regressor = model_creator(X_train)
regressor.fit(X_train, y_train, epochs=100, batch_size=32)
prediction = regressor.predict(x_test)

Epoch 1/100
30/30 [==============================] - 2s 70ms/step - loss: 4.1903e-04
Epoch 2/100
30/30 [==============================] - 2s 62ms/step - loss: 1.1893e-04
Epoch 3/100
30/30 [==============================] - 2s 64ms/step - loss: 1.3196e-04
Epoch 4/100
30/30 [==============================] - 2s 67ms/step - loss: 1.0649e-04
Epoch 5/100
30/30 [==============================] - 2s 63ms/step - loss: 9.8490e-05
Epoch 6/100
30/30 [==============================] - 2s 62ms/step - loss: 8.7017e-05
Epoch 7/100
30/30 [==============================] - 2s 61ms/step - loss: 1.0796e-04
Epoch 8/100
30/30 [==============================] - 2s 62ms/step - loss: 1.0921e-04
Epoch 9/100
30/30 [==============================] - 2s 61ms/step - loss: 1.0668e-04
Epoch 10/100
30/30 [==============================] - 2s 60ms/step - loss: 8.4008e-05
Epoch 11/100
30/30 [==============================] - 2s 61ms/step - loss: 7.9054e-05
Epoch 12/100
30/30 [==============================] - 2s 60ms/s

30/30 [==============================] - 2s 62ms/step - loss: 3.7012e-05
Epoch 97/100
30/30 [==============================] - 2s 62ms/step - loss: 4.0344e-05
Epoch 98/100
30/30 [==============================] - 2s 61ms/step - loss: 3.9653e-05
Epoch 99/100
30/30 [==============================] - 2s 62ms/step - loss: 2.8981e-05
Epoch 100/100
30/30 [==============================] - 2s 62ms/step - loss: 3.6366e-05


In [14]:
mape(prediction,totaldf_scaled[1000:])

0.37101468801712895

In [15]:
regressor = model_creator(X_train)
regressor.fit(X_train, y_train, epochs=500, batch_size=32)
prediction = regressor.predict(x_test)

Epoch 1/500
30/30 [==============================] - 2s 65ms/step - loss: 3.7544e-04
Epoch 2/500
30/30 [==============================] - 2s 64ms/step - loss: 1.5569e-04
Epoch 3/500
30/30 [==============================] - 2s 64ms/step - loss: 1.4844e-04
Epoch 4/500
30/30 [==============================] - 2s 64ms/step - loss: 1.2675e-04
Epoch 5/500
30/30 [==============================] - 2s 63ms/step - loss: 9.7950e-05
Epoch 6/500
30/30 [==============================] - 2s 63ms/step - loss: 1.0243e-04
Epoch 7/500
30/30 [==============================] - 2s 64ms/step - loss: 1.0323e-04
Epoch 8/500
30/30 [==============================] - 2s 63ms/step - loss: 8.3308e-05
Epoch 9/500
30/30 [==============================] - 2s 63ms/step - loss: 9.8290e-05
Epoch 10/500
30/30 [==============================] - 2s 64ms/step - loss: 9.4281e-05
Epoch 11/500
30/30 [==============================] - 2s 67ms/step - loss: 9.0451e-05
Epoch 12/500
30/30 [==============================] - 2s 68ms/s

30/30 [==============================] - 2s 61ms/step - loss: 3.2400e-05
Epoch 97/500
30/30 [==============================] - 2s 61ms/step - loss: 3.3672e-05
Epoch 98/500
30/30 [==============================] - 2s 61ms/step - loss: 3.8463e-05
Epoch 99/500
30/30 [==============================] - 2s 62ms/step - loss: 4.0607e-05
Epoch 100/500
30/30 [==============================] - 2s 62ms/step - loss: 5.3078e-05
Epoch 101/500
30/30 [==============================] - 2s 61ms/step - loss: 4.2336e-05
Epoch 102/500
30/30 [==============================] - 2s 61ms/step - loss: 3.5734e-05
Epoch 103/500
30/30 [==============================] - 2s 61ms/step - loss: 3.5657e-05
Epoch 104/500
30/30 [==============================] - 2s 61ms/step - loss: 3.6518e-05
Epoch 105/500
30/30 [==============================] - 2s 62ms/step - loss: 3.6283e-05
Epoch 106/500
30/30 [==============================] - 2s 61ms/step - loss: 3.7359e-05
Epoch 107/500
30/30 [==============================] - 2s 61

30/30 [==============================] - 2s 61ms/step - loss: 2.6349e-05
Epoch 191/500
30/30 [==============================] - 2s 61ms/step - loss: 2.8711e-05
Epoch 192/500
30/30 [==============================] - 2s 61ms/step - loss: 2.6433e-05
Epoch 193/500
30/30 [==============================] - 2s 61ms/step - loss: 3.7251e-05
Epoch 194/500
30/30 [==============================] - 2s 61ms/step - loss: 3.2379e-05
Epoch 195/500
30/30 [==============================] - 2s 62ms/step - loss: 3.1578e-05
Epoch 196/500
30/30 [==============================] - 2s 62ms/step - loss: 2.9413e-05
Epoch 197/500
30/30 [==============================] - 2s 62ms/step - loss: 3.0753e-05
Epoch 198/500
30/30 [==============================] - 2s 63ms/step - loss: 2.8403e-05
Epoch 199/500
30/30 [==============================] - 2s 65ms/step - loss: 2.2978e-05
Epoch 200/500
30/30 [==============================] - 2s 65ms/step - loss: 2.6278e-05
Epoch 201/500
30/30 [==============================] - 2s

30/30 [==============================] - 2s 65ms/step - loss: 2.5003e-05
Epoch 285/500
30/30 [==============================] - 2s 65ms/step - loss: 2.1137e-05
Epoch 286/500
30/30 [==============================] - 2s 64ms/step - loss: 2.6170e-05
Epoch 287/500
30/30 [==============================] - 2s 65ms/step - loss: 2.4070e-05
Epoch 288/500
30/30 [==============================] - 2s 64ms/step - loss: 3.0607e-05
Epoch 289/500
30/30 [==============================] - 2s 64ms/step - loss: 2.5683e-05
Epoch 290/500
30/30 [==============================] - 2s 64ms/step - loss: 3.1020e-05
Epoch 291/500
30/30 [==============================] - 2s 65ms/step - loss: 2.3768e-05
Epoch 292/500
30/30 [==============================] - 2s 64ms/step - loss: 2.4951e-05
Epoch 293/500
30/30 [==============================] - 2s 71ms/step - loss: 2.3661e-05
Epoch 294/500
30/30 [==============================] - 2s 72ms/step - loss: 2.5802e-05
Epoch 295/500
30/30 [==============================] - 2s

30/30 [==============================] - 2s 68ms/step - loss: 2.0075e-05
Epoch 379/500
30/30 [==============================] - 2s 69ms/step - loss: 2.1390e-05
Epoch 380/500
30/30 [==============================] - 2s 67ms/step - loss: 2.3680e-05
Epoch 381/500
30/30 [==============================] - 2s 68ms/step - loss: 2.5407e-05
Epoch 382/500
30/30 [==============================] - 2s 68ms/step - loss: 2.2088e-05
Epoch 383/500
30/30 [==============================] - 2s 67ms/step - loss: 4.6191e-05
Epoch 384/500
30/30 [==============================] - 2s 68ms/step - loss: 1.7667e-05
Epoch 385/500
30/30 [==============================] - 2s 68ms/step - loss: 2.2349e-05
Epoch 386/500
30/30 [==============================] - 2s 68ms/step - loss: 2.9226e-05
Epoch 387/500
30/30 [==============================] - 2s 67ms/step - loss: 1.8486e-05
Epoch 388/500
30/30 [==============================] - 2s 68ms/step - loss: 1.7440e-05
Epoch 389/500
30/30 [==============================] - 2s

30/30 [==============================] - 2s 67ms/step - loss: 1.9774e-05
Epoch 473/500
30/30 [==============================] - 2s 67ms/step - loss: 1.6170e-05
Epoch 474/500
30/30 [==============================] - 2s 67ms/step - loss: 1.5725e-05
Epoch 475/500
30/30 [==============================] - 2s 68ms/step - loss: 2.4519e-05
Epoch 476/500
30/30 [==============================] - 2s 74ms/step - loss: 2.9994e-05
Epoch 477/500
30/30 [==============================] - 2s 68ms/step - loss: 1.9870e-05
Epoch 478/500
30/30 [==============================] - 2s 67ms/step - loss: 2.3952e-05
Epoch 479/500
30/30 [==============================] - 2s 67ms/step - loss: 2.0803e-05
Epoch 480/500
30/30 [==============================] - 2s 68ms/step - loss: 2.2255e-05
Epoch 481/500
30/30 [==============================] - 2s 68ms/step - loss: 2.5374e-05
Epoch 482/500
30/30 [==============================] - 2s 69ms/step - loss: 2.3395e-05
Epoch 483/500
30/30 [==============================] - 2s

In [17]:
mape(prediction,totaldf_scaled[1000:])

0.5328792236758877

In [18]:
regressor = model_creator(X_train)
regressor.fit(X_train, y_train, epochs=1000, batch_size=32)
prediction = regressor.predict(x_test)

Epoch 1/1000
30/30 [==============================] - 2s 73ms/step - loss: 3.5720e-04
Epoch 2/1000
30/30 [==============================] - 2s 75ms/step - loss: 1.2723e-04
Epoch 3/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.0437e-04
Epoch 4/1000
30/30 [==============================] - 2s 74ms/step - loss: 1.0046e-04
Epoch 5/1000
30/30 [==============================] - 2s 72ms/step - loss: 8.4867e-05
Epoch 6/1000
30/30 [==============================] - 2s 74ms/step - loss: 1.0421e-04
Epoch 7/1000
30/30 [==============================] - 2s 73ms/step - loss: 8.9646e-05
Epoch 8/1000
30/30 [==============================] - 2s 72ms/step - loss: 9.5766e-05
Epoch 9/1000
30/30 [==============================] - 2s 73ms/step - loss: 9.5116e-05
Epoch 10/1000
30/30 [==============================] - 2s 78ms/step - loss: 1.0025e-04
Epoch 11/1000
30/30 [==============================] - 2s 80ms/step - loss: 9.4051e-05
Epoch 12/1000
30/30 [==============================]

30/30 [==============================] - 3s 86ms/step - loss: 3.1506e-05
Epoch 96/1000
30/30 [==============================] - 2s 81ms/step - loss: 2.9696e-05
Epoch 97/1000
30/30 [==============================] - 3s 85ms/step - loss: 3.6163e-05
Epoch 98/1000
30/30 [==============================] - 3s 85ms/step - loss: 3.6679e-05
Epoch 99/1000
30/30 [==============================] - 2s 82ms/step - loss: 3.3747e-05
Epoch 100/1000
30/30 [==============================] - 2s 80ms/step - loss: 3.3635e-05
Epoch 101/1000
30/30 [==============================] - 3s 84ms/step - loss: 3.2625e-05
Epoch 102/1000
30/30 [==============================] - 2s 83ms/step - loss: 4.0485e-05
Epoch 103/1000
30/30 [==============================] - 2s 80ms/step - loss: 3.5775e-05
Epoch 104/1000
30/30 [==============================] - 2s 80ms/step - loss: 3.2580e-05
Epoch 105/1000
30/30 [==============================] - 3s 87ms/step - loss: 4.4305e-05
Epoch 106/1000
30/30 [=============================

30/30 [==============================] - 2s 79ms/step - loss: 3.4669e-05
Epoch 189/1000
30/30 [==============================] - 2s 74ms/step - loss: 2.6304e-05
Epoch 190/1000
30/30 [==============================] - 2s 73ms/step - loss: 2.1374e-05
Epoch 191/1000
30/30 [==============================] - 2s 72ms/step - loss: 2.4670e-05
Epoch 192/1000
30/30 [==============================] - 2s 71ms/step - loss: 2.8057e-05
Epoch 193/1000
30/30 [==============================] - 2s 71ms/step - loss: 2.8614e-05
Epoch 194/1000
30/30 [==============================] - 2s 75ms/step - loss: 2.9845e-05
Epoch 195/1000
30/30 [==============================] - 2s 74ms/step - loss: 2.2760e-05
Epoch 196/1000
30/30 [==============================] - 2s 76ms/step - loss: 2.0681e-05
Epoch 197/1000
30/30 [==============================] - 2s 74ms/step - loss: 2.8425e-05
Epoch 198/1000
30/30 [==============================] - 2s 73ms/step - loss: 2.4211e-05
Epoch 199/1000
30/30 [=========================

30/30 [==============================] - 2s 71ms/step - loss: 2.8131e-05
Epoch 282/1000
30/30 [==============================] - 2s 75ms/step - loss: 2.9025e-05
Epoch 283/1000
30/30 [==============================] - 2s 76ms/step - loss: 2.4323e-05
Epoch 284/1000
30/30 [==============================] - 2s 75ms/step - loss: 2.8255e-05
Epoch 285/1000
30/30 [==============================] - 2s 76ms/step - loss: 2.5601e-05
Epoch 286/1000
30/30 [==============================] - 2s 73ms/step - loss: 2.1866e-05
Epoch 287/1000
30/30 [==============================] - 2s 78ms/step - loss: 2.1318e-05
Epoch 288/1000
30/30 [==============================] - 2s 79ms/step - loss: 2.8554e-05
Epoch 289/1000
30/30 [==============================] - 2s 75ms/step - loss: 2.1539e-05
Epoch 290/1000
30/30 [==============================] - 2s 77ms/step - loss: 2.7188e-05
Epoch 291/1000
30/30 [==============================] - 2s 70ms/step - loss: 2.1347e-05
Epoch 292/1000
30/30 [=========================

30/30 [==============================] - 2s 73ms/step - loss: 2.2171e-05
Epoch 375/1000
30/30 [==============================] - 2s 72ms/step - loss: 2.2806e-05
Epoch 376/1000
30/30 [==============================] - 2s 71ms/step - loss: 2.4076e-05
Epoch 377/1000
30/30 [==============================] - 2s 75ms/step - loss: 2.1425e-05
Epoch 378/1000
30/30 [==============================] - 2s 72ms/step - loss: 2.8300e-05
Epoch 379/1000
30/30 [==============================] - 2s 75ms/step - loss: 1.8118e-05
Epoch 380/1000
30/30 [==============================] - 2s 75ms/step - loss: 2.2915e-05
Epoch 381/1000
30/30 [==============================] - 2s 74ms/step - loss: 2.4224e-05
Epoch 382/1000
30/30 [==============================] - 2s 74ms/step - loss: 2.2880e-05
Epoch 383/1000
30/30 [==============================] - 2s 74ms/step - loss: 2.2793e-05
Epoch 384/1000
30/30 [==============================] - 2s 72ms/step - loss: 2.3600e-05
Epoch 385/1000
30/30 [=========================

...

30/30 [==============================] - 2s 69ms/step - loss: 2.1343e-05
Epoch 561/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.6834e-05
Epoch 562/1000
30/30 [==============================] - 2s 69ms/step - loss: 1.9497e-05
Epoch 563/1000
30/30 [==============================] - 2s 70ms/step - loss: 2.0046e-05
Epoch 564/1000
30/30 [==============================] - 2s 70ms/step - loss: 2.4053e-05
Epoch 565/1000
30/30 [==============================] - 2s 70ms/step - loss: 2.0599e-05
Epoch 566/1000
30/30 [==============================] - 2s 71ms/step - loss: 2.0852e-05
Epoch 567/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.4384e-05
Epoch 568/1000
30/30 [==============================] - 2s 69ms/step - loss: 2.1073e-05
Epoch 569/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.8783e-05
Epoch 570/1000
30/30 [==============================] - 2s 70ms/step - loss: 2.1196e-05
Epoch 571/1000
30/30 [=========================

30/30 [==============================] - 2s 69ms/step - loss: 2.2806e-05
Epoch 654/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.5052e-05
Epoch 655/1000
30/30 [==============================] - 2s 69ms/step - loss: 2.4281e-05
Epoch 656/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.6092e-05
Epoch 657/1000
30/30 [==============================] - 2s 69ms/step - loss: 1.5347e-05
Epoch 658/1000
30/30 [==============================] - 2s 69ms/step - loss: 1.9414e-05
Epoch 659/1000
30/30 [==============================] - 2s 69ms/step - loss: 2.3055e-05
Epoch 660/1000
30/30 [==============================] - 2s 71ms/step - loss: 1.9305e-05
Epoch 661/1000
30/30 [==============================] - 2s 71ms/step - loss: 2.1547e-05
Epoch 662/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.9186e-05
Epoch 663/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.6097e-05
Epoch 664/1000
30/30 [=========================

30/30 [==============================] - 2s 70ms/step - loss: 2.0504e-05
Epoch 747/1000
30/30 [==============================] - 2s 69ms/step - loss: 1.9545e-05
Epoch 748/1000
30/30 [==============================] - 2s 69ms/step - loss: 1.6758e-05
Epoch 749/1000
30/30 [==============================] - 2s 70ms/step - loss: 2.3085e-05
Epoch 750/1000
30/30 [==============================] - 2s 69ms/step - loss: 1.8190e-05
Epoch 751/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.6465e-05
Epoch 752/1000
30/30 [==============================] - 2s 69ms/step - loss: 1.9141e-05
Epoch 753/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.6684e-05
Epoch 754/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.8948e-05
Epoch 755/1000
30/30 [==============================] - 2s 70ms/step - loss: 2.2703e-05
Epoch 756/1000
30/30 [==============================] - 2s 67ms/step - loss: 2.1345e-05
Epoch 757/1000
30/30 [=========================

30/30 [==============================] - 2s 76ms/step - loss: 1.7570e-05
Epoch 840/1000
30/30 [==============================] - 2s 76ms/step - loss: 1.6645e-05
Epoch 841/1000
30/30 [==============================] - 2s 75ms/step - loss: 2.0199e-05
Epoch 842/1000
30/30 [==============================] - 2s 73ms/step - loss: 1.8052e-05
Epoch 843/1000
30/30 [==============================] - 2s 71ms/step - loss: 1.7283e-05
Epoch 844/1000
30/30 [==============================] - 2s 74ms/step - loss: 1.6917e-05
Epoch 845/1000
30/30 [==============================] - 2s 71ms/step - loss: 1.5686e-05
Epoch 846/1000
30/30 [==============================] - 2s 76ms/step - loss: 1.6518e-05
Epoch 847/1000
30/30 [==============================] - 2s 74ms/step - loss: 1.5330e-05
Epoch 848/1000
30/30 [==============================] - 2s 78ms/step - loss: 1.7697e-05
Epoch 849/1000
30/30 [==============================] - 2s 72ms/step - loss: 1.7141e-05
Epoch 850/1000
30/30 [=========================

30/30 [==============================] - 2s 75ms/step - loss: 1.4480e-05
Epoch 933/1000
30/30 [==============================] - 2s 72ms/step - loss: 1.8139e-05
Epoch 934/1000
30/30 [==============================] - 2s 74ms/step - loss: 1.9248e-05
Epoch 935/1000
30/30 [==============================] - 2s 72ms/step - loss: 1.7321e-05
Epoch 936/1000
30/30 [==============================] - 2s 70ms/step - loss: 1.6148e-05
Epoch 937/1000
30/30 [==============================] - 2s 72ms/step - loss: 1.7461e-05
Epoch 938/1000
30/30 [==============================] - 2s 72ms/step - loss: 1.6413e-05
Epoch 939/1000
30/30 [==============================] - 2s 70ms/step - loss: 2.2121e-05
Epoch 940/1000
30/30 [==============================] - 2s 72ms/step - loss: 1.8676e-05
Epoch 941/1000
30/30 [==============================] - 2s 76ms/step - loss: 1.7287e-05
Epoch 942/1000
30/30 [==============================] - 2s 75ms/step - loss: 2.3663e-05
Epoch 943/1000
30/30 [=========================

In [19]:
mape(prediction,totaldf_scaled[1000:])

7.8743594508834915